<a href="https://colab.research.google.com/github/jameschapman19/cca_zoo/blob/master/tutorial_notebooks/cca_zoo_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A tutorial comparing the train and test correlations of different models on MNIST data

In [ ]:
!pip install cca-zoo[deep,probabilistic]

In [ ]:
# Imports
import numpy as np
from cca_zoo.data import Noisy_MNIST_Dataset
import itertools
import matplotlib.pyplot as plt
from torch.utils.data import Subset
from torch import optim
from cca_zoo.deepmodels import objectives, architectures, DeepWrapper, DCCA,DCCA_NOI,DVCCA,DCCAE,DTCCA

# Load MNIST Data
N = 500
dataset = Noisy_MNIST_Dataset(mnist_type='FashionMNIST', train=True)
ids = np.arange(min(2 * N, len(dataset)))
np.random.shuffle(ids)
train_ids, val_ids = np.array_split(ids, 2)
val_dataset = Subset(dataset, val_ids)
train_dataset = Subset(dataset, train_ids)
test_dataset = Noisy_MNIST_Dataset(mnist_type='FashionMNIST', train=False)
test_ids = np.arange(min(N, len(test_dataset)))
np.random.shuffle(test_ids)
test_dataset = Subset(test_dataset, test_ids)
train_view_1, train_view_2, train_rotations, train_OH_labels, train_labels = train_dataset.dataset.to_numpy(
    train_dataset.indices)
val_view_1, val_view_2, val_rotations, val_OH_labels, val_labels = val_dataset.dataset.to_numpy(val_dataset.indices)
test_view_1, test_view_2, test_rotations, test_OH_labels, test_labels = test_dataset.dataset.to_numpy(
    test_dataset.indices)

# Settings

# The number of latent dimensions across models
latent_dims = 2
# The number of folds used for cross-validation/hyperparameter tuning
cv_folds = 5
# For running hyperparameter tuning in parallel (0 if not)
jobs = 2
# Number of iterations for iterative algorithms
max_iter = 2
# number of epochs for deep models
epochs = 50

# Canonical Correlation Analysis

In [ ]:
from cca_zoo.models import CCA, CCA_ALS
"""
### Linear CCA by eigendecomposition
"""
linear_cca = CCA(latent_dims=latent_dims)

linear_cca.fit(train_view_1, train_view_2)

linear_cca_results = np.stack(
    (linear_cca.score(train_view_1, train_view_2), linear_cca.score(test_view_1, test_view_2)))

"""
### Linear CCA by alternating least squares (can pass more than 2 views)
"""

linear_cca_als = CCA_ALS(latent_dims=latent_dims)

linear_cca_als.fit(train_view_1, train_view_2)

linear_cca_als_results = np.stack(
    (linear_cca_als.score(train_view_1, train_view_2), linear_cca_als.score(test_view_1, test_view_2)))

# Partial Least Squares


In [ ]:
from cca_zoo.models import PLS
"""
### PLS with scikit-learn (only permits 2 views)
"""
pls = PLS(latent_dims=latent_dims)

pls.fit(train_view_1, train_view_2)

pls_results = np.stack(
    (pls.score(train_view_1, train_view_2), pls.score(test_view_1, test_view_2)))

# Extension to multiple views



In [ ]:
from cca_zoo.models import GCCA, MCCA
"""
### (Regularized) Generalized CCA(can pass more than 2 views)
"""
train_view_3=train_view_1+np.random.rand(train_view_1.shape)
test_view_3=test_view_1+np.random.rand(test_view_1.shape)

# small ammount of regularisation added since data is not full rank
c=[0.5,0.5,0.5]

gcca = GCCA(latent_dims=latent_dims,c=c)

gcca.fit(train_view_1, train_view_2,train_view_3)

gcca_results = np.stack((gcca.score(train_view_1, train_view_2, train_view_3), gcca.score(test_view_1, test_view_2, test_view_3)))

"""
### (Regularized) Multiset CCA(can pass more than 2 views)
"""

mcca = MCCA(latent_dims=latent_dims, c=c)

mcca.fit(train_view_1, train_view_2,train_view_1)

mcca_results = np.stack((mcca.score(train_view_1, train_view_2, train_view_3), mcca.score(test_view_1, test_view_2, test_view_3)))

"""
### Multiset CCA by alternating least squares
"""
mcca_als = CCA_ALS(latent_dims=latent_dims, max_iter=max_iter)

mcca_als.fit(train_view_1, train_view_2,train_view_3)

mcca_als_results = np.stack(
    (mcca_als.score(train_view_1, train_view_2, train_view_3), mcca_als.score(test_view_1, test_view_2, test_view_3)))

"""
### Multiset PLS by alternating least squares
"""
mcca_pls = PLS(latent_dims=latent_dims, max_iter=max_iter)

mcca_pls.fit(train_view_1, train_view_2,train_view_1)

mcca_pls_results = np.stack(
    (mcca_als.score(train_view_1, train_view_2, train_view_3), mcca_pls.score(test_view_1, test_view_2, test_view_3)))

# Tensor CCA

In [ ]:
from cca_zoo.models import TCCA
"""
### (Regularized) Tensor CCA(can pass more than 2 views)
"""

tcca = TCCA(latent_dims=latent_dims, c=c)

#memory requirement for tensor is massive so take first 100 features
tcca.fit(train_view_1[:,:100], train_view_2[:,:100],train_view_3[:,:100])

tcca_results = np.stack((tcca.score(train_view_1[:,:100], train_view_2[:,:100], train_view_3[:,:100]), tcca.score(test_view_1[:,:100], test_view_2[:,:100], test_view_3[:,:100])))

# Weighted GCCA/Missing Observation GCCA

In [ ]:
#observation_matrix
K = np.ones((3, N))
K[0, 200:] = 0
K[1, :100] = 0

#view weights
view_weights=[1,2,1.2]

c=[0.5,0.5,0.5]

gcca = GCCA(latent_dims=latent_dims,c=c,view_weights=view_weights)

gcca.fit(train_view_1, train_view_2,train_view_1,K=K)

gcca_results = np.stack((gcca.score(train_view_1, train_view_2), gcca.score(test_view_1, test_view_2)))

# Rgularised CCA solutions based on alternating minimisation/alternating least squares

We implement Witten's penalized matrix decomposition form of sparse CCA using 'pmd'

We implement Waaijenborg's penalized CCA using elastic net using 'elastic'

We implement Mai's sparse CCA using 'scca'

Furthermore, any of these methods can be extended to multiple views. Witten describes this method explicitly.

In [ ]:
from cca_zoo.models import rCCA, PMD,SCCA,ElasticCCA
"""
### Ridge CCA (can pass more than 2 views)
"""
c1 = [0.1, 0.3, 0.7, 0.9]
c2 = [0.1, 0.3, 0.7, 0.9]
param_candidates = {'c': list(itertools.product(c1, c2))}

ridge = rCCA(latent_dims=latent_dims).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True, jobs=jobs,
    plot=True)

ridge_results = np.stack((ridge.score(train_view_1,train_view_2), ridge.score(test_view_1, test_view_2)))

"""
### Sparse CCA (Penalized Matrix Decomposition) (can pass more than 2 views)
"""

# PMD
c1 = [1, 3, 7, 9]
c2 = [1, 3, 7, 9]
param_candidates = {'c': list(itertools.product(c1, c2))}

pmd = PMD(latent_dims=latent_dims, max_iter=max_iter).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True, jobs=jobs,
    plot=True)

pmd_results = np.stack((pmd.score(train_view_1,train_view_2), pmd.score(test_view_1, test_view_2)))

"""
### Sparse CCA (can pass more than 2 views)
"""

# Sparse CCA
c1 = [0.00001, 0.0001]
c2 = [0.00001, 0.0001]
param_candidates = {'c': list(itertools.product(c1, c2))}

scca = SCCA(latent_dims=latent_dims, max_iter=max_iter).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True,
    jobs=jobs, plot=True)

scca_results = np.stack(
    (scca.score(train_view_1,train_view_2), scca.score(test_view_1, test_view_2)))


"""
### Elastic CCA (can pass more than 2 views)
"""

# Elastic CCA
c1 = [0.001, 0.0001]
c2 = [0.001, 0.0001]
l1_1 = [0.01, 0.1]
l1_2 = [0.01, 0.1]
param_candidates = {'c': list(itertools.product(c1, c2)), 'l1_ratio': list(itertools.product(l1_1, l1_2))}

elastic = ElasticCCA(latent_dims=latent_dims,
                              max_iter=max_iter).gridsearch_fit(train_view_1,
                                                                train_view_2,
                                                                param_candidates=param_candidates,
                                                                folds=cv_folds,
                                                                verbose=True,
                                                                jobs=jobs,
                                                                plot=True)

elastic_results = np.stack(
    (elastic.score(train_view_1,train_view_2), elastic.score(test_view_1, test_view_2)))

# Kernel CCA

In [ ]:
from cca_zoo.models import KCCA
"""
### Kernel CCA

Similarly, we can use kernel CCA methods with [method='kernel']

We can use different kernels and their associated parameters in a similar manner to before
- regularized linear kernel CCA: parameters :  'kernel'='linear', 0<'c'<1
- polynomial kernel CCA: parameters : 'kernel'='poly', 'degree', 0<'c'<1
- gaussian rbf kernel CCA: parameters : 'kernel'='gaussian', 'sigma', 0<'c'<1
"""
# %%
# r-kernel cca
c1 = [0.9, 0.99]
c2 = [0.9, 0.99]

param_candidates = {'kernel': [['linear', 'linear']], 'c': list(itertools.product(c1, c2))}

kernel_reg = KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                          folds=cv_folds,
                                                          param_candidates=param_candidates,
                                                          verbose=True, jobs=jobs,
                                                          plot=True)
kernel_reg_results = np.stack((
    kernel_reg.score(train_view_1,train_view_2),
    kernel_reg.score(test_view_1, test_view_2)[0, 1, :]))

# kernel cca (poly)
degree1 = [2, 3]
degree2 = [2, 3]

param_candidates = {'kernel': [['poly', 'poly']], 'degree': list(itertools.product(degree1, degree2)),
                    'c': list(itertools.product(c1, c2))}

kernel_poly = KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                           folds=cv_folds,
                                                           param_candidates=param_candidates,
                                                           verbose=True, jobs=jobs,
                                                           plot=True)

kernel_poly_results = np.stack((
    kernel_poly.score(train_view_1,train_view_2),
    kernel_poly.score(test_view_1, test_view_2)[0, 1, :]))

# kernel cca (gaussian)
gamma1 = [1e+1, 1e+2, 1e+3]
gamma2 = [1e+1, 1e+2, 1e+3]

param_candidates = {'kernel': [['rbf', 'rbf']], 'gamma': list(itertools.product(gamma1, gamma2)),
                    'c': list(itertools.product(c1, c2))}

kernel_gaussian = KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                               folds=cv_folds,
                                                               param_candidates=param_candidates,
                                                               verbose=True, jobs=jobs,
                                                               plot=True)

kernel_gaussian_results = np.stack((
    kernel_gaussian.score(train_view_1,train_view_2),
    kernel_gaussian.score(test_view_1, test_view_2)[0, 1, :]))

# Deep CCA

DCCA can be optimized using Andrew's original tracenorm objective or Wang's DCCA by nonlinear orthogonal iterations using the argument als=True.

In [ ]:
"""
### Deep Learning

We also have deep CCA methods (and autoencoder variants)
- Deep CCA (DCCA)
- Deep Canonically Correlated Autoencoders (DCCAE)

We introduce a Config class from configuration.py. This contains a number of default settings for running 

"""

# %%
# DCCA
print('DCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dcca_model = DeepWrapper(dcca_model)

dcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dcca_results = np.stack((dcca_model.score(train_dataset), dcca_model.score(test_dataset)))

# DCCA_NOI
print('DCCA by non-linear orthogonal iterations')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_noi_model = DCCA_NOI(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dcca_noi_model = DeepWrapper(dcca_noi_model)

dcca_noi_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dcca_noi_results = np.stack(
    (dcca_noi_model.score(train_dataset), dcca_noi_model.score(test_dataset)))

# DCCA with custom optimizers and schedulers

In [ ]:
# DCCA
dcca_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2],
                                objective=objectives.CCA)
optimizer = optim.Adam(dcca_model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 1)
# hidden_layer_sizes are shown explicitly but these are also the defaults
dcca_model = DeepWrapper(dcca_model)
dcca_model.fit(train_dataset, val_dataset=val_dataset,epochs=20)

# DGCCA and DMCCA for more than 2 views

The only change we need to make is to the objective argument to perform DGCCA and DMCCA.

In [ ]:
# DGCCA
print('DGCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dgcca_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], objective=objectives.GCCA)

dgcca_model = DeepWrapper(dgcca_model)

dgcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dgcca_results = np.stack(
    (dgcca_model.score(train_dataset), dgcca_model.score(test_dataset)))

# DMCCA
print('DMCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dmcca_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], objective=objectives.MCCA)

dmcca_model = DeepWrapper(dmcca_model)

dmcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dmcca_results = np.stack(
    (dmcca_model.score(train_dataset), dmcca_model.score(test_dataset)))

# Deep Canonically Correlated Autoencoders
We need to add decoders in order to model deep canonically correlated autoencoders and we also use the DCCAE class which inherits from DCCA

In [ ]:
# DCCAE
print('DCCAE')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
decoder_1 = architectures.Decoder(latent_dims=latent_dims, feature_size=784)
decoder_2 = architectures.Decoder(latent_dims=latent_dims, feature_size=784)
dccae_model = DCCAE(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2])

dccae_model = DeepWrapper(dccae_model)

#can also pass a tuple of numpy arrays
dccae_model.fit((train_view_1, train_view_2), epochs=epochs)

dccae_results = np.stack(
    (dccae_model.score(train_dataset), dccae_model.score(test_dataset)))

# Deep Variational CCA

In [ ]:
"""
### Deep Variational Learning
Finally we have Deep Variational CCA methods.
- Deep Variational CCA (DVCCA)
- Deep Variational CCA - private (DVVCA_p)

These are both implemented by the DVCCA class with private=True/False and both_encoders=True/False. If both_encoders,
the encoder to the shared information Q(z_shared|x) is modelled for both x_1 and x_2 whereas if both_encoders is false
it is modelled for x_1 as in the paper
"""

# %%
# DVCCA (technically bi-DVCCA)
print('DVCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
decoder_1 = architectures.Decoder(latent_dims=latent_dims, feature_size=784, norm_output=True)
decoder_2 = architectures.Decoder(latent_dims=latent_dims, feature_size=784, norm_output=True)
dvcca_model = DVCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2])

dvcca_model = DeepWrapper(dvcca_model)

dvcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dvcca_model_results = np.stack(
    (dvcca_model.score(train_dataset), dvcca_model.score(test_dataset)))

# DVCCA_private (technically bi-DVCCA_private)
print('DVCCA_private')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
private_encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
private_encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
decoder_1 = architectures.Decoder(latent_dims=latent_dims * 2, feature_size=784, norm_output=True)
decoder_2 = architectures.Decoder(latent_dims=latent_dims * 2, feature_size=784, norm_output=True)
dvccap_model = DVCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2],
                           private_encoders=[private_encoder_1, private_encoder_2])

dvccap_model = DeepWrapper(dvccap_model)

dvccap_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dvccap_model_results = np.stack(
    (dvccap_model.score(train_dataset), dvccap_model.score(test_dataset)))

# Convolutional Deep CCA (and using other architectures)
We provide a standard CNN encoder and decoder but users can build their own encoders and decoders by inheriting BaseEncoder and BaseDecoder for seamless integration with the pipeline

In [ ]:
print('Convolutional DCCA')
encoder_1 = architectures.CNNEncoder(latent_dims=latent_dims, channels=[3, 3])
encoder_2 = architectures.CNNEncoder(latent_dims=latent_dims, channels=[3, 3])
dcca_conv_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dcca_conv_model = DeepWrapper(dcca_conv_model)

# to change the models used change the cfg.encoder_models. We implement a CNN_Encoder and CNN_decoder as well
# as some based on brainnet architecture in cca_zoo.architectures. Equally you could pass your own encoder/decoder models

dcca_conv_model.fit((train_view_1.reshape((-1, 1, 28, 28)), train_view_2.reshape((-1, 1, 28, 28))), epochs=epochs)

dcca_conv_results = np.stack(
    (dcca_conv_model.score((test_view_1.reshape((-1, 1, 28, 28)),test_view_2.reshape((-1, 1, 28, 28))), dcca_conv_model.score((test_view_1.reshape((-1, 1, 28, 28)),test_view_2.reshape((-1, 1, 28, 28)))))))

# DTCCA

In [ ]:
# %%
# DTCCA
print('DTCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dtcca_model = DTCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dtcca_model = DeepWrapper(dtcca_model)

dtcca_model.fit(train_dataset, val_dataset=val_dataset, epochs=epochs)

dtcca_results = np.stack((dtcca_model.score(train_dataset), dtcca_model.score(test_dataset)))

# Generate Some Plots

In [ ]:
"""
### Make results plot to compare methods
"""
# %%

all_results = np.stack(
    [linear_cca_results, gcca_results, mcca_results, pls_results, pmd_results, elastic_results,
     scca_results, kernel_reg_results, kernel_poly_results,
     kernel_gaussian_results, dcca_results, dgcca_results, dmcca_results, dvcca_model_results,
     dcca_conv_results, dtcca_results],
    axis=0)
all_labels = ['linear', 'gcca', 'mcca', 'pls', 'pmd', 'elastic', 'scca', 'linear kernel', 'polynomial kernel',
              'gaussian kernel', 'deep CCA', 'deep generalized CCA', 'deep multiset CCA', 'deep VCCA',
              'deep convolutional cca', 'DTCCA']

from cca_zoo.utils import plot_utils

plot_utils.plot_results(all_results, all_labels)
plt.show()